# Data Preprocessing

# Setup

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import seaborn as sns
colours = sns.color_palette("Set2")
from sentence_transformers import SentenceTransformer, util
import warnings
warnings.filterwarnings("ignore")


project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)
from utils import get_table_from_supabase

#get keys from env
load_dotenv()
url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")

In [1]:
funder_cols = ["registered_num", "name", 
               "activities", "objectives", "objectives_activities", "achievements_performance", "grant_policy",
               "ext_areas", "ext_bens", "ext_causes",
               "is_nua"
               ]

In [ ]:
recipient_cols = ["recipient_id", "name", 
                  "recipient_activities", "recipient_objectives",
                  "recip_ext_areas", "recip_ext_bens", "recip_ext_causes",
                  ]

In [ ]:
#check name and actitivities for giveaway single-beneficiary words
name_check = funders_df["name"].str.contains(
    r'SCHOOL|FRIENDS OF|PTA|CHURCH|HOSPITAL',
    case=False,
    na=False
)
activities_check = funders_df["activities"].str.contains(
    r'PUPIL?',
    case=False,
    na=False
)

funders_df["is_potential_sbf"] = name_check | activities_check

print(f"Flagged as potential SBFs: {funders_df['is_potential_sbf'].sum()}")

# Feature Engineering